# **Computational Drug Discovery [Part 1] Download Bioactivity Data**

Chanin Nantasenamat

[*'Data Professor' YouTube channel*](http://youtube.com/dataprofessor)

In this Jupyter notebook, we will be building a real-life **data science project** that you can include in your **data science portfolio**. Particularly, we will be building a machine learning model using the ChEMBL bioactivity data.

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [10]:
! pip install chembl_webresource_client # had to use "py -m pip install chembl_webresource_client" in cmd to install on 3.11.4 

## **Importing libraries**

In [115]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client # initiates a new client connection to the ChEMBL web services

## **Search for Target protein**

### **Target search for coronavirus**

In [116]:
# Target search for coronavirus
target = new_client.target # creates a new target object
target_query = target.search('coronavirus') # searches for the target coronavirus
targets = pd.DataFrame.from_dict(target_query) # converts the target_query into a pandas dataframe
targets # displays the dataframe

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Coronavirus,Coronavirus,17.0,False,CHEMBL613732,[],ORGANISM,11119
1,[],SARS coronavirus,SARS coronavirus,14.0,False,CHEMBL612575,[],ORGANISM,227859
2,[],Feline coronavirus,Feline coronavirus,14.0,False,CHEMBL612744,[],ORGANISM,12663
3,[],Murine coronavirus,Murine coronavirus,14.0,False,CHEMBL5209664,[],ORGANISM,694005
4,[],Human coronavirus 229E,Human coronavirus 229E,12.0,False,CHEMBL613837,[],ORGANISM,11137
5,[],Human coronavirus OC43,Human coronavirus OC43,12.0,False,CHEMBL5209665,[],ORGANISM,31631
6,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,10.0,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859
7,[],Middle East respiratory syndrome-related coron...,Middle East respiratory syndrome-related coron...,9.0,False,CHEMBL4296578,[],ORGANISM,1335626
8,"[{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...",SARS coronavirus,Replicase polyprotein 1ab,4.0,False,CHEMBL5118,"[{'accession': 'P0C6X7', 'component_descriptio...",SINGLE PROTEIN,227859
9,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,4.0,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049


### **Select and retrieve bioactivity data for *SARS coronavirus 3C-like proteinase* (fifth entry)**

We will assign the fifth entry (which corresponds to the target protein, *coronavirus 3C-like proteinase*) to the ***selected_target*** variable 

In [117]:
selected_target = targets.target_chembl_id[6] # selects the target with the index 6 - SARS coronavirus 3C-like proteinase
selected_target # displays the selected target ID

'CHEMBL3927'

Here, we will retrieve only bioactivity data for *coronavirus 3C-like proteinase* (CHEMBL3927) that are reported as IC$_{50}$ values in nM (nanomolar) unit.

In [118]:
activity = new_client.activity # creates a new activity object
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50") # filters the activity object for the selected target and the standard type IC50

In [119]:
df = pd.DataFrame.from_dict(res)

In [120]:
df.head(3)

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1480935,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,7.2
1,None,None,1480936,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,9.4
2,None,None,1481061,[],CHEMBL830868,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,13.5


In [121]:
df.standard_type.unique()

array(['IC50'], dtype=object)

**NOTES by CA**: We have seen that the standard_value in this dataset is always IC50, but that's not alway the case. When that happens, we need to filter the standard_type column to only include IC50 values. Also, we need to ensure the standard_units column is in nM. Standard_value = concentration, so corresponds to the potency of the drug, the lower the value, the more potent the drug is. In order to elicit 50% of the inhibition of a target protein, you would need lower concentrations of the drug. Therefore, the lower the standard_value value, the more potent the drug is.

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [122]:
df.to_csv('bioactivity_data.csv', index=False) # saves the dataframe as a csv file

**NOTES by CA**: Skipped this part as using Github instead

## **Handling missing data & Converting units**
If any compounds have missing value for the **standard_value** column then drop it.  
If the **units** are not uniform, then convert all to nM. (by CA)  

In [123]:
df2 = df[df.standard_value.notna()] # filters the data frame for standard values that are not NaN and stores is as df2
df2

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1480935,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,7.2
1,None,None,1480936,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,9.4
2,None,None,1481061,[],CHEMBL830868,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,13.5
3,None,None,1481065,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,13.11
4,None,None,1481066,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,None,None,12041507,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,10.6
129,None,None,12041508,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,10.1
130,None,None,12041509,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,11.5
131,None,None,12041510,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,10.7


In [124]:
# Data frame with no NA values in the standard_value column
df2.head(3)

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1480935,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,7.2
1,None,None,1480936,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,9.4
2,None,None,1481061,[],CHEMBL830868,In vitro inhibitory concentration against SARS...,B,None,None,BAO_0000190,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,None,None,IC50,uM,UO_0000065,None,13.5


Apparently, for this dataset there is no missing data. But we can use the above code cell for bioactivity data of other target protein.

Next we can check the units for the bioactivity data that we have so that we can do the appropriate conversion (if required). **by CA**

In [132]:
# Checking if the column units has different measuring units
df2.units.unique()

array(['uM', None, 'nM'], dtype=object)

In [133]:
# Remove None values 
df3 = df2[df2.units.notna()] # filters the data frame for standard units that are not NaN and stores is as df3

In [134]:
# Check AGAIN if the units column 
df3.units.unique()

array(['uM', 'nM'], dtype=object)

In [135]:
# Check type of standard_value 
df3.standard_value.dtype

dtype('O')

In [136]:
# Convert standard_value from object to float before converting units
df3 = df3.copy()
df3['standard_value'] = pd.to_numeric(df3['standard_value'])

In [137]:
# check AGAIN type of standard_value to make sure it is what we want
df3.standard_value.dtype

dtype('float64')

In [138]:
# CONVERTING STANDARD_VALUE TO nM

# Create a dictionary with the units and their conversion factors to nM
unit_conversion = {'nM': 1, 'uM': 1000, 'pM': 0.001, 'mM': 1000000, 'M': 1000000000}

# Create a function that converts the standard_value to nM
def unit_converter(unit, value):
    return unit_conversion[unit] * value

# Create a new column that contains the converted standard_value
df4 = df3.copy()
df4['standard_value_nM'] = df4.apply(lambda x: unit_converter(x['units'], x['standard_value']), axis=1)

# check columns standard_value, standard_value_nM and units
df4[['standard_value', 'standard_value_nM', 'units']]

,standard_value,standard_value_nM,units
0,7200.0,7200000.0,uM
1,9400.0,9400000.0,uM
2,13500.0,13500000.0,uM
3,13110.0,13110000.0,uM
4,2000.0,2000000.0,uM
...,...,...,...
128,10600.0,10600000.0,uM
129,10100.0,10100000.0,uM
130,11500.0,11500000.0,uM
131,10700.0,10700000.0,uM


## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

When evaluating IC50 values (often given in nM or µM), a lower IC50 indicates greater potency of the compound, as it requires less of the compound to achieve 50% inhibition. Conversely, a higher IC50 indicates lesser potency.

However, the specific concentration ranges that are considered "active," "inactive," and "intermediate" can vary based on the target, the specific assay being used, the therapeutic area, and the literature or guidelines that a researcher or organization decides to follow. But a common general classification, especially in the context of drug discovery for anti-infectives or anti-cancer agents, might be:

    Active: IC50 values < 1,000 nM (or < 1 µM)
    Intermediate: IC50 values between 1,000 nM and 10,000 nM (or between 1 µM and 10 µM)
    Inactive: IC50 values > 10,000 nM (or > 10 µM)

These thresholds, however, are general. It's crucial to consult specific literature, guidelines, or expert consensus relevant to the particular biological target or therapeutic area in question.

(by CA)

In [140]:
bioactivity_class = []
for value in df4.standard_value_nM:
  if float(value) >= 10000:
    bioactivity_class.append("inactive")
  elif float(value) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Create a data frame with "molecule_chEMBL_id", "canonical_smiles", "standard_value_nM" and "bioactivity_class" columns**

In [161]:
# Adding columns from df4 'molecule_chembl_id', 'canonical_smiles', 'standard_value_nM'
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value_nM']
df5 = df4[selection]
df5

,molecule_chembl_id,canonical_smiles,standard_value_nM
0,CHEMBL187579,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,7200000.0
1,CHEMBL188487,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,9400000.0
2,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,13500000.0
3,CHEMBL426082,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,13110000.0
4,CHEMBL187717,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],2000000.0
...,...,...,...
128,CHEMBL2146517,COC(=O)[C@@]1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(C)c...,10600000.0
129,CHEMBL187460,C[C@H]1COC2=C1C(=O)C(=O)c1c2ccc2c1CCCC2(C)C,10100000.0
130,CHEMBL363535,Cc1coc2c1C(=O)C(=O)c1c-2ccc2c(C)cccc12,11500000.0
131,CHEMBL227075,Cc1cccc2c3c(ccc12)C1=C(C(=O)C3=O)[C@@H](C)CO1,10700000.0


In [176]:
# Adding column 'bioactivity_class' that assigns the level of bioactivity
df6 = pd.concat([df5, pd.Series(bioactivity_class, name='bioactivity_class')], axis=1)
df6



,molecule_chembl_id,canonical_smiles,standard_value_nM,bioactivity_class
0,CHEMBL187579,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,7200000.0,inactive
1,CHEMBL188487,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,9400000.0,inactive
2,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,13500000.0,inactive
3,CHEMBL426082,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,13110000.0,inactive
4,CHEMBL187717,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],2000000.0,inactive
...,...,...,...,...
68,NaN,NaN,NaN,inactive
70,NaN,NaN,NaN,inactive
72,NaN,NaN,NaN,active
74,NaN,NaN,NaN,active


In [179]:
# Show only the first 5 rows where the bioactivity_class is active
df6[df6.bioactivity_class == 'active'].head(5)


,molecule_chembl_id,canonical_smiles,standard_value_nM,bioactivity_class
71,CHEMBL208763,O=C(CSc1nccc(-c2cc(-c3ccccc3)no2)n1)Nc1ccc(Cl)cc1,15000000.0,active
73,CHEMBL209227,Cc1nc(-c2nc(-c3ccnc(SCC(=O)Nc4ccc(Cl)cc4)n3)cs...,14000000.0,active
75,CHEMBL210092,CSc1sc(-c2nc(C)cs2)c(C)c1-c1ccnc(SCC(=O)Nc2ccc...,11000000.0,active
77,CHEMBL377150,Cn1nc(-c2ccc(-c3ccnc(SCC(=O)Nc4ccc(Cl)cc4)n3)s...,10000000.0,active
78,CHEMBL212454,O=C(Oc1ccc(S(=O)(=O)c2ccc(OC(=O)C(Cl)=C(Cl)Cl)...,900000.0,active


In [181]:
# Count number of active
df6.bioactivity_class.value_counts()

bioactivity_class
inactive    97
active       8
Name: count, dtype: int64

Saves dataframe to CSV file

In [164]:
df5.to_csv('bioactivity_preprocessed_data.csv', index=False)

---